<font size="6">Import the libraries

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

<font size="6">Load the model

In [2]:
loaded_model = tf.saved_model.load("Saved_Model_With_TF1")
loaded_model_keras = tf.keras.models.load_model("Saved_Model_With_TFWith_Keras")
loaded_model_keras_h5 = tf.keras.models.load_model('MobileNetWithoutTheLast6Layers.h5', custom_objects={'KerasLayer':hub.KerasLayer})
# loaded_model.summary()

<h1>Convert to TFLITE Cells<h1\>

<font size="6">Float-16 Quantization

In [ ]:
# Passing the Keras model to the TF Lite Converter.
converter = tf.lite.TFLiteConverter.from_saved_model("Saved_Model_With_TF1")
 
# Using float-16 quantization.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
 
# Converting the model.
tflite_fp16_model = converter.convert()
 
# Saving the model.
with open('Fp_16_model.tflite', 'wb') as f:
  f.write(tflite_fp16_model)

<font size="6">Dynamic Range Quantization " convert to 8-bit precision values"

In [ ]:
# Create the .tflite file

tflite_model_file = "model.tflite"
converter = tf.lite.TFLiteConverter.from_saved_model("Saved_Model_With_TF1")

#bigbly hna warning eno bifks l Optimize for size
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]  
tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

<font size="6">Full integar quantization

<h1> Clustering </h1>

In [ ]:
loaded_model_keras.summary()

In [ ]:

import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 32,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

# Cluster a whole model
clustered_model = cluster_weights(loaded_model_keras, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

In [ ]:
# Helper function uses `cluster_weights` to make only 
# the Dense layers train with clustering
def apply_clustering_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return cluster_weights(layer, **clustering_params)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_clustering_to_dense` 
# to the layers of the model.
clustered_model = tf.keras.models.clone_model(
    loaded_model_keras,
    clone_function=apply_clustering_to_dense,
)

clustered_model.summary()

In [ ]:
loaded_model_keras.summary()
clustered_model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy'])

In [4]:
pixels = 224
IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")

BATCH_SIZE = 16#@param {type:"integer"}

normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])


data_dir = 'D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train'
def build_dataset(subset):
  return tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=.20,
      subset=subset,
      label_mode="categorical",
      # Seed needs to provided when using validation_split and shuffle = True.
      # A fixed seed is used so that the validation set is stable across runs.
      seed=123,
      image_size=IMAGE_SIZE,
      batch_size=1)

train_ds = build_dataset("training")
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()

train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

val_ds = build_dataset("validation")
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

Input size (224, 224)
Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.


In [ ]:
steps_per_epoch = train_size  // BATCH_SIZE
validation_steps = valid_size // BATCH_SIZE

clustered_model.fit(
  x = train_ds,
  batch_size=500,
  epochs= 1,
  verbose= 1,
  validation_data=val_ds,
  steps_per_epoch=steps_per_epoch,
  validation_steps=validation_steps
  )

In [ ]:
saved_model_path = "Clustered_Saved_Model_With_TF1"
tf.saved_model.save(clustered_model, saved_model_path)
clustered_model.save("Clustered_MobileNetWithoutTheLast6Layers.h5")
print("Saved Model")
saved_model_path = "Clustered_Saved_Model_With_TFWith_Keras"
tf.keras.models.save_model(clustered_model, saved_model_path)

<h1>Proning

In [5]:
import tensorflow_model_optimization as tfmot
loaded_model_keras_h5.load_weights('PretrainedWeights.h5')
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
# validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = 69600  #param int
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.60,
                                                               final_sparsity=0.90,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(loaded_model_keras_h5, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 prune_low_magnitude_conv1 (  (None, 112, 112, 32)     1730      
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_conv1_b  (None, 112, 112, 32)     129       
 n (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_conv1_r  (None, 112, 112, 32)     1         
 elu (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_conv_dw  (None, 112, 112, 32)     289       
 _1 (PruneLowMagnitude)                                      

In [6]:
import tempfile
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer='adam',
      metrics=['accuracy']
)

steps_per_epoch = train_size  // BATCH_SIZE
validation_steps = valid_size // BATCH_SIZE

model_for_pruning.fit(
  x = train_ds,
  batch_size=BATCH_SIZE,
  epochs= 2,
  verbose= 1,
  validation_data=val_ds,
  steps_per_epoch=steps_per_epoch,
  validation_steps=validation_steps,
  callbacks= callbacks
  )

Epoch 1/2
4350/4350 [==============================] - 3051s 696ms/step - loss: 0.3627 - accuracy: 0.9031 - val_loss: 0.1290 - val_accuracy: 0.9703
Epoch 2/2
4350/4350 [==============================] - 3071s 706ms/step - loss: 0.1034 - accuracy: 0.9797 - val_loss: 0.0992 - val_accuracy: 0.9799


In [10]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

print("final model")
model_for_export.summary()

final model
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                          

In [14]:
saved_model_path = "1_Pruned_Exported__Saved_Model_With_TF1"
tf.saved_model.save(model_for_export, saved_model_path)
model_for_export.save("1_Pruned_Exported_MobileNetWithoutTheLast6Layers.h5")
print("Saved Model")
saved_model_path = "1_Pruned_Exported__Saved_Model_With_TFWith_Keras"
tf.keras.models.save_model(model_for_export, saved_model_path)

INFO:tensorflow:Assets written to: 1_Pruned_Exported__Saved_Model_With_TF1\assets


INFO:tensorflow:Assets written to: 1_Pruned_Exported__Saved_Model_With_TF1\assets


Saved Model


INFO:tensorflow:Assets written to: 1_Pruned_Exported__Saved_Model_With_TFWith_Keras\assets


INFO:tensorflow:Assets written to: 1_Pruned_Exported__Saved_Model_With_TFWith_Keras\assets


Convert it now to TFLite

In [3]:
# Create the .tflite file

tflite_model_file = "3_withoutlast9Layers_Model.tflite"
converter = tf.lite.TFLiteConverter.from_saved_model("test/Saved_Model_With_TF1_without_the_last_9_layers")

#bigbly hna warning eno bifks l Optimize for size
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]  
tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)